## Bibliotecas a Utilizar

In [7]:
import numpy as np
import pandas as pd
from os import scandir

## Enlaces

## Trabajando los Data Frames - Función cleanData()
* Creando una columna con fecha para cada Data Frame
* Creando columna con variable de Riesgos de cada dato 
* Reemplazando los datos incorrectos
* Eliminando columnas que no sirven
* Posibilidad de guardar en archivo csv

## Función completa

In [17]:
def g_archivos(ruta = './data/Indicadores_Gestion/'):
    """ Generate the list with the path files of all files """
    return sorted([archivo.path for archivo in scandir(ruta) if archivo.is_file()],reverse=True)


def enlace_df(l_enlaces):
    """ Generate a list with all the Data Frames """
    l_dataframes = []
    for enlace in l_enlaces:
        df = pd.read_excel(enlace, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
        l_dataframes.append(df)
    
    return l_dataframes


def tirar_resetear(df):
    """ Throw the useless rows and reset the index """
    df.drop(df.index[[0,1,2,3,4,5]],inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


def columna_fechas(df, fecha):
    """ Create and fill 'Fecha' column """
    fechas = ['2018-06', '2018-03',
        '2017-12', '2017-09', '2017-06', '2017-03',
        '2016-12', '2016-09', '2016-06', '2016-03',
        '2015-12', '2015-09', '2015-06', '2015-03',
        '2014-12', '2014-09', '2014-06', '2014-03',
        '2013-12', '2013-09', '2013-06', '2013-03',
        '2012-12', '2012-09', '2012-06', '2012-03',]
    
    df['Fecha'] = fechas[fecha]
    return df


def identificador_ramo(df):
    """ Fill 'Categoría' column with the apropiate insurance branch """
    df['Categoría'] = None
    current_cat = 'Patrimoniales y Mixtas'
    
    for i in range(len(df)):
        if pd.isnull(df.iloc[i].NJ):
            current_cat = df.iloc[i]['Denominación']
        else:
            df.loc[i,'Categoría'] = current_cat
    
    current_cat = 'Patrimoniales y Mixtas'
    return df


def no_nan(df):
    """ Eliminate the wrong values """
    df.replace('/',0,inplace=True)
    df.replace('//',0,inplace=True)
    df.replace('///',0,inplace=True)
    df.fillna(0)
    x = ['%','\n','[',']',' ']
    for vaule in range(len(x)):
        df.columns = df.columns.str.replace(x[vaule],'')
    return df


def columna_mercado(df):
    """ Create the '% Mercado' column and calculate the correct % for each branch """
    l_categorias = df['Categoría'].unique()
    df['% Mercado'] = None
    new_df = pd.DataFrame()
    
    for categoria in l_categorias:
        tmp = df[df['Categoría'] == categoria].copy()
        p_total_mercado = tmp['ProducciónTotal'].sum()
        tmp['Mercado'] = tmp['ProducciónTotal'].map(lambda x: round(x*100/p_total_mercado,2))
        new_df = new_df.append(tmp)
        
    return new_df


def a_categoria(df):
    """ Convert 'Categoría' and 'Denominación' columns in category type, and eliminate the useless rows """
    df['Categoría'] = df['Categoría'].astype('category', inplace=True)
    df['Denominación'] = df['Denominación'].astype('category', inplace=True)
    df = df[~df.Categoría.isnull()]
    df.reset_index(drop=True, inplace=True) 
    return df


def filtro_ramo(df, ramo):
    """ Filtered df by insurance branch """
    ramos = {'PAT':'Patrimoniales y Mixtas', 
             'ART':'Riesgos del Trabajo', 
             'TRANSPORTE':'Transporte Público Pasajeros', 
             'VIDA':'Seguros de Vida', 
             'RETIRO':'Seguros de Retiro'}
    
    return df[ df['Categoría']== ramos[ramo]]


def guardado(df, nombre_archivo):
    """ Save in csv format the filtered info """
    df.to_csv(nombre_archivo)
    
    
def cleanData(ramo):
    """ Call al the funtions to get the DataFrame clean and filtered by branch """
    ramo = ramo.upper()
    d_nombre_archivo = {'PAT':'01_Patrimoniales_y_Mixtas', 
                         'ART':'02_Riesgos_del_Trabajo', 
                         'TRANSPORTE':'03_Transporte_Público_Pasajeros', 
                         'VIDA':'04_Seguros_de_Vida', 
                         'RETIRO':'05_Seguros_de_Retiro'}
    nombre_archivo = './clean_data/Indicadores_Gestion/{}.csv'.format(d_nombre_archivo[ramo])
    l_enlaces = g_archivos()             # U: Generate the list with the path files of all files
    l_dataframes = enlace_df(l_enlaces)  # U: Generate a list with all the Data Frames
    largo = len(l_dataframes)            # U: Get the leng of the list for iterate 
    df_Unido = pd.DataFrame()            # U: Create de variable of the future Data Frame
    

    for l, df in zip(range(largo),l_dataframes):
        tirar_resetear(df)               # U: Throw the useless rows and reset the index
        columna_fechas(df,l)             # U: Create and fill 'Fecha' column 
        identificador_ramo(df)           # U: Fill 'Categoría' column with the apropiate insurance branch
        no_nan(df)                       # U: Eliminate the wrong values
        df = columna_mercado(df)         # U: Agregate the % of the Market that each company have in his branch
        a_categoria(df)                  # U: Convert 'Categoría' and 'Denominación' columns in category type, and eliminate the useless rows
        
        df_Unido = df_Unido.append(filtro_ramo(df, ramo), ignore_index=True) # U: Append filtered df by insurance branch       

    guardado(df_Unido, nombre_archivo)   # U: Save in csv the filtered info
    return 


In [18]:
#cleanData('RetirO')

In [ ]:
ramos = ['PAT', 'ArT','TransPorte','viDa','RetirO']
for ramo in ramos:
    cleanData(ramo)

 **Posibles ramos a elegir en la función:**
* 'PAT' = Patrimoñales y Mixta
* 'ART' = Riesgos del Trabajo
* 'TRANSPORTE' = Transporte Público Pasajeros
* 'VIDA' = Seguros de Vida
* 'RETIRO' = Seguros de Retiro

In [1]:
d_nombre_archivo = {'PAT':'01_Patrimoniales_y_Mixtas', 
                         'ART':'02_Riesgos_del_Trabajo', 
                         'TRANSPORTE':'03_Transporte_Público_Pasajeros', 
                         'VIDA':'04_Seguros_de_Vida', 
                         'RETIRO':'05_Seguros_de_Retiro'}

d_nombre_archivo['PAT']

'01_Patrimoniales_y_Mixtas'

In [5]:
'Ramo'

'RAMO'